In [4]:
import shutil
import sys
import os
from pathlib import Path
import json
import pysubs2
from pysubs2 import SSAFile, SSAEvent, make_time

In [2]:
movienet_path = Path('/home/jianghui/dataset/MovieNet/')
movienet_subset_path = Path('/home/jianghui/dataset/MovieNet-Subset/')

In [5]:
srt_path = Path(movienet_path, "subtitle1K.v1/subtitle")
example_srt = Path(srt_path, "tt0047396.srt")

In [6]:
srt_file = SSAFile.load(example_srt)

In [48]:
print(pysubs2.time.ms_to_frames(srt_file[0].start, 23.98))
print(pysubs2.time.ms_to_frames(srt_file[0].end, 23.98))
print(srt_file[0].plaintext)

3530
3568
Men, are you over 40?


In [9]:
60 / 24

2.5

In [18]:
shot_path = Path(movienet_path, 'movie1K.shot_detection.v1/shot')
example_shot = Path(shot_path, "tt0047396.txt")

In [43]:
shot_list = []
with open(example_shot, "r") as f_shot:
    for line in f_shot:
        line_list = line.split()
        start_frame = int(line_list[0])
        end_frame = int(line_list[1])
        shot_list.append([start_frame, end_frame])
shot_list


[[0, 48],
 [49, 329],
 [330, 2538],
 [2539, 3482],
 [3483, 3805],
 [3806, 6285],
 [6286, 6537],
 [6538, 6581],
 [6582, 6741],
 [6742, 6942],
 [6943, 7136],
 [7137, 7170],
 [7171, 7238],
 [7239, 7385],
 [7386, 7625],
 [7626, 7701],
 [7702, 7770],
 [7771, 7879],
 [7880, 8122],
 [8123, 8190],
 [8191, 8314],
 [8315, 8358],
 [8359, 8526],
 [8527, 8627],
 [8628, 8760],
 [8761, 8796],
 [8797, 9070],
 [9071, 9239],
 [9240, 9889],
 [9890, 9994],
 [9995, 10053],
 [10054, 10814],
 [10815, 11027],
 [11028, 11088],
 [11089, 11232],
 [11233, 11284],
 [11285, 11863],
 [11864, 11903],
 [11904, 11959],
 [11960, 12030],
 [12031, 12059],
 [12060, 12091],
 [12092, 12212],
 [12213, 12272],
 [12273, 13125],
 [13126, 13495],
 [13496, 13551],
 [13552, 13572],
 [13573, 14123],
 [14124, 14350],
 [14351, 16252],
 [16253, 16437],
 [16438, 16620],
 [16621, 16660],
 [16661, 16741],
 [16742, 16845],
 [16846, 16966],
 [16967, 17777],
 [17778, 18290],
 [18291, 18567],
 [18568, 18918],
 [18919, 19039],
 [19040, 19226],

In [67]:
srt_list = []
for srt in srt_file:
    start_frame = pysubs2.time.ms_to_frames(srt.start, 23.98)
    end_frame = pysubs2.time.ms_to_frames(srt.end, 23.98)
    srt_element = [start_frame, end_frame, srt.plaintext.replace("\n"," ")]
    srt_list.append(srt_element)
srt_list


[[3530, 3568, 'Men, are you over 40?'],
 [3572,
  3634,
  'When you wake up in the morning, do you feel tired and rundown?'],
 [3638, 3702, 'Do you have that listless feeling?'],
 [6041, 6093, '- Jefferies. - Congratulations, Jeff.'],
 [6097, 6163, '- For what? - Getting rid of that cast.'],
 [6167, 6201, 'Who said I was getting rid of it?'],
 [6205, 6251, 'This is Wednesday,'],
 [6255, 6363, 'Seven weeks from the day you broke your leg, Yes or no?'],
 [6367, 6425, 'Gunnison, how did you get to be such a big editor'],
 [6429, 6475, 'with such a small memory?'],
 [6479, 6529, 'Thrift, industry and hard work...'],
 [6533, 6604, 'and catching the publisher with his secretary,'],
 [6608, 6658, '- did I get the wrong day? - No,'],
 [6681, 6712, 'No, wrong week.'],
 [6716, 6835, 'Next Wednesday I emerge from this... plaster cocoon.'],
 [6839,
  6952,
  "That's too bad, Jeff. Well, I guess I can't be lucky every day."],
 [6956, 6987, 'Forget I called,'],
 [6991, 7053, 'Yeah, I sure feel sorry

In [70]:
from distutils.log import error


shot_list_format = []
srt_index = 0
for shot_index, shot in enumerate(shot_list):
    tmp_sentences = []
    tmp_sentences_id = []
    for srt in srt_list[srt_index:]:
        if (srt[0] >= shot[0]) and (srt[0] <= shot[1]) and (srt[1] <= shot[1]):
            tmp_sentences.append(srt[2])
            tmp_sentences_id.append(srt_index)
            srt_index += 1
        
        if (srt[0] >= shot[0]) and (srt[0] <= shot[1]) and (srt[1] > shot[1]):
            tmp_sentences.append(srt[2])
            tmp_sentences_id.append(srt_index)

        if (srt[0] <= shot[0]) and (srt[1] >= shot[0]) and (srt[1] <= shot[1]):
            tmp_sentences.append(srt[2])
            tmp_sentences_id.append(srt_index)
            srt_index += 1
        
        if (srt[0] <= shot[0]) and (srt[1] >= shot[0]) and (srt[1] > shot[1]):
            tmp_sentences.append(srt[2])
            tmp_sentences_id.append(srt_index)
    
    shot_list_format_element = {"shot":shot_index, "sentences":tmp_sentences, "sentences_id":tmp_sentences_id}
    shot_list_format.append(shot_list_format_element)
shot_list_format       


[{'shot': 0, 'sentences': [], 'sentences_id': []},
 {'shot': 1, 'sentences': [], 'sentences_id': []},
 {'shot': 2, 'sentences': [], 'sentences_id': []},
 {'shot': 3, 'sentences': [], 'sentences_id': []},
 {'shot': 4,
  'sentences': ['Men, are you over 40?',
   'When you wake up in the morning, do you feel tired and rundown?',
   'Do you have that listless feeling?'],
  'sentences_id': [0, 1, 2]},
 {'shot': 5,
  'sentences': ['- Jefferies. - Congratulations, Jeff.',
   '- For what? - Getting rid of that cast.',
   'Who said I was getting rid of it?',
   'This is Wednesday,',
   'Seven weeks from the day you broke your leg, Yes or no?'],
  'sentences_id': [3, 4, 5, 6, 7]},
 {'shot': 6,
  'sentences': ['Seven weeks from the day you broke your leg, Yes or no?',
   'Gunnison, how did you get to be such a big editor',
   'with such a small memory?',
   'Thrift, industry and hard work...',
   'and catching the publisher with his secretary,'],
  'sentences_id': [7, 8, 9, 10, 11]},
 {'shot': 7,

In [71]:
info_path = Path(movienet_path, "movie1K.video_info.v1.json")
with open(info_path, 'r') as f_info:
    info_json = json.load(f_info)

info_json["tt0047396"]["fps"]

23.98

In [77]:
import re
example_str = "tt0047396_0002"
m = re.search(r'(?<=_)\d+', example_str)
int(m.group(0))

2

In [79]:
a = [1, 2, 3, 4, 5, 6, 7 ,8 ,9]
a[-1]

9

In [80]:
scene_path = Path(movienet_subset_path, "all/scene/", 'tt0047396' + '.json')
with open(scene_path, 'r') as f_scene:
    scene_json = json.load(f_scene)

In [82]:
scene_json

[{'scene_id': 0, 'shot_start_id': 0, 'shot_end_id': 1},
 {'scene_id': 1, 'shot_start_id': 1, 'shot_end_id': 4},
 {'scene_id': 2, 'shot_start_id': 4, 'shot_end_id': 5},
 {'scene_id': 3, 'shot_start_id': 5, 'shot_end_id': 6},
 {'scene_id': 4, 'shot_start_id': 6, 'shot_end_id': 8},
 {'scene_id': 5, 'shot_start_id': 8, 'shot_end_id': 10},
 {'scene_id': 6, 'shot_start_id': 10, 'shot_end_id': 20},
 {'scene_id': 7, 'shot_start_id': 20, 'shot_end_id': 22},
 {'scene_id': 8, 'shot_start_id': 22, 'shot_end_id': 30},
 {'scene_id': 9, 'shot_start_id': 30, 'shot_end_id': 31},
 {'scene_id': 10, 'shot_start_id': 31, 'shot_end_id': 32},
 {'scene_id': 11, 'shot_start_id': 32, 'shot_end_id': 34},
 {'scene_id': 12, 'shot_start_id': 34, 'shot_end_id': 43},
 {'scene_id': 13, 'shot_start_id': 43, 'shot_end_id': 71},
 {'scene_id': 14, 'shot_start_id': 71, 'shot_end_id': 72},
 {'scene_id': 15, 'shot_start_id': 72, 'shot_end_id': 73},
 {'scene_id': 16, 'shot_start_id': 73, 'shot_end_id': 75},
 {'scene_id': 17, 

In [84]:
for scene_json_element in scene_json:
    for shot_id in range(scene_json_element['shot_start_id'], scene_json_element['shot_end_id']+1):
        shot_list_format[shot_id]["scene_id"] = scene_json_element['scene_id']

shot_list_format

[{'shot': 0, 'sentences': [], 'sentences_id': [], 'scene_id': 0},
 {'shot': 1, 'sentences': [], 'sentences_id': [], 'scene_id': 1},
 {'shot': 2, 'sentences': [], 'sentences_id': [], 'scene_id': 1},
 {'shot': 3, 'sentences': [], 'sentences_id': [], 'scene_id': 1},
 {'shot': 4,
  'sentences': ['Men, are you over 40?',
   'When you wake up in the morning, do you feel tired and rundown?',
   'Do you have that listless feeling?'],
  'sentences_id': [0, 1, 2],
  'scene_id': 2},
 {'shot': 5,
  'sentences': ['- Jefferies. - Congratulations, Jeff.',
   '- For what? - Getting rid of that cast.',
   'Who said I was getting rid of it?',
   'This is Wednesday,',
   'Seven weeks from the day you broke your leg, Yes or no?'],
  'sentences_id': [3, 4, 5, 6, 7],
  'scene_id': 3},
 {'shot': 6,
  'sentences': ['Seven weeks from the day you broke your leg, Yes or no?',
   'Gunnison, how did you get to be such a big editor',
   'with such a small memory?',
   'Thrift, industry and hard work...',
   'and ca

In [ ]:
a = "shot_0000_img_0.jpg"
m = re.search('shot_\d+_img_1.jpg', a)
m, m.group(0)